In [1]:
!pip install accelerate peft bitsandbytes transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.4 MB/s eta 0:00:0000:0100:01


In [2]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from transformers import AutoModelForCausalLM, AutoTokenizer

# Download the NLTK punkt tokenizer models
nltk.download("punkt")

# Model name
model_name = "shivenkk/Llama-2-7b-chat-finetune"

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [8]:
# Generate text samples
prompt = "What is photosynsthesis"
text_samples = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors="pt"),
    max_length=50,  # Adjust max_length as needed
    num_return_sequences=5,  # Number of samples to generate
    temperature=0.7,  # Adjust temperature for diversity
    top_k=50,  # Adjust top_k for diversity
    top_p=0.9,  # Adjust top_p for diversity
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode generated samples
generated_texts = [tokenizer.decode(sample, skip_special_tokens=True) for sample in text_samples]

# Reference texts (manually created or heuristic method)
reference_texts = [
    "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll. It involves the conversion of carbon dioxide and water into glucose and oxygen.",
    "Photosynthesis is a vital process in which green plants convert light energy into chemical energy to produce food. Through a series of complex biochemical reactions, carbon dioxide and water are converted into glucose and oxygen.",
    "Photosynthesis is the biological process used by plants to convert light energy into chemical energy, which is stored in the form of glucose. This process releases oxygen as a byproduct and is crucial for sustaining life on Earth.",
    "Photosynthesis is a fundamental process in which green plants, algae, and some bacteria convert light energy into chemical energy stored in glucose molecules. This process not only produces food for the organism but also releases oxygen into the atmosphere.",
    "Photosynthesis is the biochemical process by which green plants and certain other organisms transform light energy into chemical energy stored in the bonds of organic molecules. Through this process, carbon dioxide and water are converted into glucose and oxygen, fueling life on Earth.",
]

# Ensure the number of hypotheses and references are the same
assert len(generated_texts) == len(reference_texts), "Number of hypotheses and references should be the same"

# Calculate BLEU score
bleu_score = corpus_bleu([[text] for text in reference_texts], generated_texts)
print("BLEU Score:", bleu_score)

BLEU Score: 0.5002444711171223
